In [1]:
# %pip install bm25s
# %pip install spacy
# %pip install -U 'spacy[cuda12x]'
# %pip install rouge_score
# %pip install pysbd

In [2]:
import functions as fct
import bm25s
import numpy as np
import json
import time
import os

resource module not available on Windows


# BM25

In [3]:
# # Définition des chemins vers les fichiers JSON d'entraînement et de développement
# train_path = 'SCOTUS/train.json'
# dev_path = 'SCOTUS/dev.json'

# # Ouverture du fichier d'entraînement
# train = fct.open_file(train_path, "json")

# # Récupération du document source du premier élément de l'ensemble d'entraînement
# # document = train[0]["raw_source"]
# document = train

# # Récupération des éléments de la cible (faits, question, conclusion)
# paragraph_target = (
#     train[0]['raw_target']['facts_of_the_case'] +
#     train[0]['raw_target']['question'] +
#     train[0]['raw_target']['conclusion']
# )

# train_path = 'data_txt_save/train_0.txt'
# train = fct.open_file(train_path, "txt")

# document = train


# # Segmentation des phrases du document source
# sentences = fct.sent_segmentation(document, method='custom_spacy')

# # Résumé des phrases en utilisant le modèle BERT
# summary = fct.bb25LegalSum(sentences, "bert-base-uncased", 5)

# # Évaluation de la qualité du résumé à l'aide de la métrique ROUGE
# bb25_rouge = fct.rouge_evaluations(" ".join(summary), paragraph_target)

In [4]:
# bb25_rouge

In [29]:
# Définition des chemins vers les fichiers JSON d'entraînement et de développement
train_path_json = 'SCOTUS/train.json'
dev_path_json = 'SCOTUS/dev.json'

# Ouverture du fichier d'entraînement JSON
train_json = fct.open_file(train_path_json, "json")

# Récupération du document source et des éléments de la cible (faits, question, conclusion)
document_json = train_json[0]["raw_source"]
paragraph_target_json = (
    train_json[0]['raw_target']['facts_of_the_case'] +
    train_json[0]['raw_target']['question'] +
    train_json[0]['raw_target']['conclusion']
)

# Définir le chemin du fichier texte à traiter
train_path_txt = 'data_txt_save/train_0.txt'
clean_path_txt = 'clean_data_txt_save/train_0.txt'

# Ouverture du fichier d'entraînement TXT
document_txt = fct.open_file(train_path_txt, "txt")
clean_document_txt = fct.open_file(clean_path_txt, "txt")

# Liste des méthodes de segmentation et des modèles à tester
methods = ['nltk', 'spacy', 'pySBD', 'custom_spacy']  # Méthodes de segmentation à tester
model = "bert-base-uncased"  # Modèle de résumé à utiliser

def save_txt(text, folder_name, file_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    file_path = os.path.join(folder_name, file_name)

    with open(file_path, 'w') as file:
        file.write(text)

# Fonction pour évaluer les modèles
def evaluate_models(document, paragraph_target, file_type):
    results = []
    for method in methods:
        start_time = time.time()

        # Segmentation des phrases du document source
        sentences = fct.sent_segmentation(document, method=method)
        
        # Résumé des phrases
        summary = fct.bb25LegalSum(sentences, model, 5)
        
        # Évaluation de la qualité du résumé à l'aide de la métrique ROUGE
        bb25_rouge = fct.rouge_evaluations(" ".join(summary), paragraph_target)

        end_time = time.time()
        execution_time = end_time - start_time
        
        # summary_conc = "\n".join(summary)
        # save_txt(summary_conc, f"summaries/{method}/", "summary_train_0.txt") #save train_0 summary
        
        results.append({'Method': file_type+'_'+method,
                        'rouge1': bb25_rouge[0],
                        'rouge2': bb25_rouge[1],
                        'rougeL': bb25_rouge[2],
                        'Execution Time': execution_time
                        })
        
    return results, summary

results = []

# Évaluation des modèles pour le fichier JSON
r, _ = evaluate_models(document_json, paragraph_target_json, "JSON")
results += r

# Évaluation des modèles pour le fichier TXT
r, txt_summary = evaluate_models(document_txt, paragraph_target_json, "TXT")  # Utilise le même paragraphe cible
results += r
r, cleaned_txt_summary = evaluate_models(clean_document_txt, paragraph_target_json, "Cleaned TXT")  # Utilise le même paragraphe cible
results += r

In [34]:
print(txt_summary)
print(cleaned_txt_summary)

["the court rejected respondent 's remaining challenges to his conviction , and the state supreme court declined review .", "the path to the jury 's guilty verdicts on the murder and attempted-murder charges was not an easy one .", 'you then apply the law to those facts as i state it to you , and you must accept and follow the law .', 'the law is right there , and i think elements of the law was [ sic ] given to you in those instructions .', '* * *', '< a > 10 < /a > per curiam', 'accordingly , the comments made and not made by the court to the jury did not coerce a particular verdict or deny packer any constitutional rights .', "that court dismissed the petition , but granted a certificate of appealability on the question whether the state trial judge violated respondent 's fourteenth amendment rights by coercing the jury into rendering a verdict on the attempted-murder and second-degree murder counts .", '< a > 5 < /a > room , and , in the presence of the attorneys and the defendant 

In [30]:
import pandas as pd

df = pd.DataFrame(results)

def highlight_min_max(df):
    styles = pd.DataFrame('', index=df.index, columns=df.columns)

    # Appliquer le style pour les colonnes 'Precision', 'Recall', 'F1-Score'
    for col in ['rouge1', 'rouge2', 'rougeL']:
        # Top 3 maximums et minimums
        top_3_max = df[col].nlargest(3)
        top_3_min = df[col].nsmallest(3)

        # Appliquer le dégradé vert pour les max
        for i in df.index:
            if df[col].iloc[i] in top_3_max.values:
                rank = top_3_max.rank(ascending=False)[top_3_max == df[col].iloc[i]].values[0]
                alpha = 1 - (rank - 1) / 3
                styles.loc[i, col]= f'background-color: rgba(50, 200, 50, {alpha});'

        # Appliquer le dégradé rouge pour les min
        for i in df.index:
            if df[col].iloc[i] in top_3_min.values:
                rank = top_3_min.rank()[top_3_min == df[col].iloc[i]].values[0]
                alpha = 1 - (rank - 1) / 3  
                styles.loc[i, col]= f'background-color: rgba(200, 50, 50, {alpha});'

    # Pour la colonne 'Execution Time', inverser les couleurs (max en rouge, min en vert)
    col = 'Execution Time'
    top_3_max = df[col].nlargest(3)
    top_3_min = df[col].nsmallest(3)

    # Appliquer le dégradé rouge pour les max
    for i in df.index:
        if df[col].iloc[i] in top_3_max.values:
            rank = top_3_max.rank(ascending=False)[top_3_max == df[col].iloc[i]].values[0]
            alpha = 1 - (rank - 1) / 3 
            styles.loc[i, col]= f'background-color: rgba(200, 50, 50, {alpha});'

    # Appliquer le dégradé vert pour les min
    for i in df.index:
        if df[col].iloc[i] in top_3_min.values:
            rank = top_3_min.rank()[top_3_min == df[col].iloc[i]].values[0]
            alpha = 1 - (rank - 1) / 3 
            styles.loc[i, col]= f'background-color: rgba(50, 200, 50, {alpha});'
            
    return styles

styled_df = df.style.apply(highlight_min_max, axis=None)

styled_df

,Method,rouge1,rouge2,rougeL,Execution Time
0,JSON_nltk,0.085658,0.023118,0.032685,13.875355
1,JSON_spacy,0.117647,0.017751,0.082353,62.144917
2,JSON_pySBD,0.141643,0.028490,0.090652,63.073558
3,JSON_custom_spacy,0.178161,0.034682,0.103448,60.064394
4,TXT_nltk,0.466102,0.165957,0.194915,8.458565
5,TXT_spacy,0.371287,0.124378,0.188119,9.822311
6,TXT_pySBD,0.368182,0.114155,0.181818,8.138017
7,TXT_custom_spacy,0.478070,0.167401,0.206140,9.750938
8,Cleaned TXT_nltk,0.229692,0.090141,0.128852,8.167267
9,Cleaned TXT_spacy,0.400966,0.126214,0.188406,9.426583


We try pySBD on 20 cleaned documents

In [31]:
txt_result = []
cleaned_txt_result = []

methods = ['pySBD']

for i in range(0, 20):

    document_json = train_json[i]["raw_source"]
    paragraph_target_json = (
        train_json[i]['raw_target']['facts_of_the_case'] +
        train_json[i]['raw_target']['question'] +
        train_json[i]['raw_target']['conclusion']
    )

    path_txt = 'data_txt_save/train_'+ str(i) + '.txt'
    document_txt = fct.open_file(path_txt, "txt")
    clean_path_txt = 'clean_data_txt_save/train_'+ str(i) + '.txt'
    clean_document_txt = fct.open_file(clean_path_txt, "txt")
    
    r, _ = evaluate_models(document_txt, paragraph_target_json, "TXT")  
    txt_result += r
    r, _ = evaluate_models(clean_document_txt, paragraph_target_json, "Cleaned TXT")  
    cleaned_txt_result += r
    
txt_df = pd.DataFrame(txt_result)  
cleaned_txt_df = pd.DataFrame(cleaned_txt_result)

mean_txt = txt_df[['rouge1', 'rouge2', 'rougeL', 'Execution Time']].mean()
mean_cleaned_txt = cleaned_txt_df[['rouge1', 'rouge2', 'rougeL', 'Execution Time']].mean()


final_df = pd.DataFrame({
    'Text': ['txt', 'cleaned txt'],
    'rouge1': [mean_txt['rouge1'], mean_cleaned_txt['rouge1']],
    'rouge2': [mean_txt['rouge2'], mean_cleaned_txt['rouge2']],
    'rougeL': [mean_txt['rougeL'], mean_cleaned_txt['rougeL']],
    'Execution Time': [mean_txt['Execution Time'], mean_cleaned_txt['Execution Time']]
})

final_df

,Text,rouge1,rouge2,rougeL,Execution Time
0,txt,0.293587,0.077637,0.152772,6.186914
1,cleaned txt,0.266624,0.069339,0.138787,5.867955
